In [2]:
import pandas as pd

In [3]:
ingested_articles = pd.read_csv("data/wrangled_articles.csv")
ingested_articles.head()

,article_id,ingestion_id,normalized_text,polled_at,published_at,embedding
0,d9509dc9-a405-4866-9201-19e3d063d137,000d8dea-8f60-470e-b539-eb51780b55db,Outrage as 'miserable' shovel-wielding neighbo...,2026-01-07 13:33:24.714064+00,2026-01-07 13:29:24+00,[-0.01143522 -0.03746951 -0.00753336 ... -0.00...
1,9a156ce2-11d6-4eff-b243-e2d2bb208faa,0072ab60-b1ed-42a9-9624-88c18292e588,"Morality, military might and a sense of mischi...",2026-01-09 05:17:56.535446+00,2026-01-09 03:15:08+00,[ 0.00615122 -0.03799464 -0.01626345 ... 0.00...
2,df3b2fc9-4d0f-4199-bca1-ea6d954e0c31,007f72ee-5039-47bb-8c61-c981f2810d3a,Republican senator vows to block all Fed nomin...,2026-01-12 15:17:02.967673+00,2026-01-12 14:55:53+00,[-0.00727808 -0.02314259 -0.02138059 ... 0.01...
3,d0f6077e-15be-497e-9a2b-1c023486ce9a,00879af4-8278-40a8-8338-dba8a0b45810,Starmer prepares to rip up Brexit: PM ready to...,2026-01-05 04:42:39.19293+00,2026-01-04 15:49:23+00,[ 0.01808163 0.02099675 -0.01054433 ... 0.00...
4,847a6cef-0c95-4a0f-9388-7345ba01bf26,00937788-6ba8-4db8-8107-645754dd7362,ANOTHER poll shows Labour in third behind Refo...,2026-01-07 11:12:56.5161+00,2026-01-07 10:57:35+00,[ 0.03266984 -0.00281101 -0.01202355 ... 0.01...
